In [1]:
# ssh
import os
import paramiko
from tqdm import tqdm
import stat

# torch
import torch
# plot
import pandas as pd

# local dependencies
from utility.Control import load_config, cfg
from utility.DataLoader import get_data_loaders
from utility.EverythingNeeded import build_model, get_item_from_dataloader, convert_batch_to_df

In [2]:
### Only for training remotely

# SSH credentials and remote CSV file path
# host = 'lxslc7.ihep.ac.cn'
# username = os.environ.get('IHEP_USERNAME')
# password = os.environ.get('IHEP_PASSWORD')

host = 'bl-0.inpac.sjtu.edu.cn'
username = os.environ.get('SJTU_USERNAME')
password = os.environ.get('SJTU_PASSWORD')

# Establish an SSH connection
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, username=username, password=password)
print(f'Successfully connected to {username}@{host}')

# Download to local
# Download the file using SFTP
sftp_client = ssh.open_sftp()
print(f'Open sftp to {username}@{host}')


def download(remote_path, local_path):
    # Get the file or folder attributes
    remote_attr = sftp_client.stat(remote_path)
    remote_file_size = remote_attr.st_size

    # Check if the path is a file or directory
    if stat.S_ISDIR(remote_attr.st_mode):
        # If it's a directory, create it locally if it doesn't exist
        if not os.path.exists(local_path):
            os.makedirs(local_path)

        # List the files and folders in the remote directory
        for file in sftp_client.listdir_attr(remote_path):
            download(os.path.join(remote_path, file.filename), os.path.join(local_path, file.filename))
    else:
        # If it's a file, download it
        os.makedirs(os.path.dirname(local_path), exist_ok=True)
        with tqdm(
                total=remote_file_size, unit='B', unit_scale=True, desc=os.path.basename(remote_path)
        ) as progress_bar:
            sftp_client.get(
                remote_path, local_path,
                callback=lambda current, _: progress_bar.update(current - progress_bar.n)
            )


# Define the remote and local paths

remote_project_base = '/lustre/collider/zhangyulei/DeepLearning/Tracking/workspace/'
local_base = '/Users/avencast/PycharmProjects/trkgnn/workspace/visible'

paths = [
    (
        f'{remote_project_base}/mpnn.yaml',
        f'{local_base}/mpnn.yaml'
    ),
    (
        f'{remote_project_base}/output.rec.new/summaries_0.csv',
        f'{local_base}/output.rec.new/summaries_0.csv'
    ),
    (
        f'{remote_project_base}/output.rec.new/model.checkpoints/model_checkpoint_070.pth.tar',
        f'{local_base}/output.rec.new/model.checkpoints/model_checkpoint_070.pth.tar'
    ),
]

# Download each file and folder
for remote_path, local_path in paths:
    download(remote_path, local_path)

# Close the SFTP and SSH clients
sftp_client.close()
ssh.close()

Successfully connected to zhangyulei@bl-0.inpac.sjtu.edu.cn
Open sftp to zhangyulei@bl-0.inpac.sjtu.edu.cn


mpnn.yaml: 100%|██████████| 2.25k/2.25k [00:00<00:00, 27.3kB/s]
summaries_0.csv: 100%|██████████| 79.4k/79.4k [00:00<00:00, 603kB/s]
model_checkpoint_070.pth.tar: 100%|██████████| 714k/714k [00:00<00:00, 1.76MB/s]


In [3]:
# local_base = '/Users/avencast/PycharmProjects/trkgnn/workspace/bepc2'
config_path = paths[0][1]
summary = paths[1][1]
model_path = paths[2][1]

In [4]:
# load config
load_config(config_path)
# load model
model = build_model('cpu', False)
tar = torch.load(model_path, map_location=torch.device('cpu'))
model.load_state_dict(tar['model'])

<All keys matched successfully>

In [26]:

cfg['data']['read_from_graph']=False
# load data
data_loader = get_data_loaders(
    input_dir=r'/Users/avencast/CLionProjects/darkshine-simulation/workspace/Tracker_GNN.root',
    chunk_size="500 MB",
    batch_size=50,
    shuffle=False,
)
loader = next(data_loader)

# load batch for index 0
batch = get_item_from_dataloader(loader, 10)
# send it to model
batch_output = model(batch)
batch_output = torch.sigmoid(batch_output)

df = convert_batch_to_df(batch, ['x', 'y', 'z', 'By'])
df['predict'] = pd.DataFrame(batch_output.detach().cpu().numpy(), columns=['predict'])

df_edge_value = pd.concat([df['edge'], df['y'], df['predict']], axis=1)
merged_df = df_edge_value.merge(df['node'], left_on='start', right_index=True)

df_edge = merged_df.merge(df['node'], left_on='end', right_index=True, suffixes=('_start', '_end'))
df_node = df['node']

## Visualization



In [27]:
from scripts.plotting import plot_xyz_plotly

# Usage example
plot_xyz_plotly(df_node, df_edge, threshold=0.3)
# plot_xyz_plotly(df_node, df_edge, threshold=0.2)


In [28]:
from scripts.plotting import plot_xyz_plotly_3d

# Usage example
plot_xyz_plotly_3d(df_node, df_edge, threshold=0.5)


In [6]:
display(df_edge)

,start,end,truth,predict,x_start,y_start,z_start,x_end,y_end,z_end
0,0,21,0.0,5.715887e-07,-6.735,0.0,-607.672485,-5.295,0.0,-507.672485
20,1,21,0.0,2.499768e-09,6.045,0.0,-607.672485,-5.295,0.0,-507.672485
40,2,21,0.0,2.181598e-08,-8.715,0.0,-607.672485,-5.295,0.0,-507.672485
60,3,21,0.0,6.316399e-06,-2.865,0.0,-607.672485,-5.295,0.0,-507.672485
80,4,21,0.0,1.456391e-07,-1.695,0.0,-607.672485,-5.295,0.0,-507.672485
...,...,...,...,...,...,...,...,...,...,...
2379,117,141,0.0,1.876460e-11,10.305,0.0,-107.672501,5.925,0.0,-7.672500
2399,118,141,0.0,1.832707e-09,-2.835,0.0,-107.672501,5.925,0.0,-7.672500
2419,119,141,0.0,2.563927e-06,-1.935,0.0,-107.672501,5.925,0.0,-7.672500
2439,120,141,0.0,2.125535e-02,6.795,0.0,-107.672501,5.925,0.0,-7.672500
